In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)


In [2]:
 ! pip install -q kaggle

In [3]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
100% 609M/612M [00:07<00:00, 40.4MB/s]
100% 612M/612M [00:07<00:00, 82.9MB/s]


In [4]:
!unzip gtsrb-german-traffic-sign.zip -d ./traffic_sign

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: ./traffic_sign/train/5/00005_00053_00010.png  
  inflating: ./traffic_sign/train/5/00005_00053_00011.png  
  inflating: ./traffic_sign/train/5/00005_00053_00012.png  
  inflating: ./traffic_sign/train/5/00005_00053_00013.png  
  inflating: ./traffic_sign/train/5/00005_00053_00014.png  
  inflating: ./traffic_sign/train/5/00005_00053_00015.png  
  inflating: ./traffic_sign/train/5/00005_00053_00016.png  
  inflating: ./traffic_sign/train/5/00005_00053_00017.png  
  inflating: ./traffic_sign/train/5/00005_00053_00018.png  
  inflating: ./traffic_sign/train/5/00005_00053_00019.png  
  inflating: ./traffic_sign/train/5/00005_00053_00020.png  
  inflating: ./traffic_sign/train/5/00005_00053_00021.png  
  inflating: ./traffic_sign/train/5/00005_00053_00022.png  
  inflating: ./traffic_sign/train/5/00005_00053_00023.png  
  inflating: ./traffic_sign/train/5/00005_00053_00024.png  
  inflating: ./traffic_sign/train/5/00005_00053_00025

In [16]:
data_dir = '/content/traffic_sign/'
train_path = '/content/traffic_sign/Train/'
test_path = '/content/traffic_sign/Test/'


IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

In [17]:
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

43

In [18]:

classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing veh over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Veh > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing veh > 3.5 tons' }

In [19]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)

image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

(39209, 30, 30, 3) (39209,)


In [20]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

In [21]:
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.2, random_state=42, shuffle=True)

X_train = X_train/255
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

X_train.shape (31367, 30, 30, 3)
X_valid.shape (7842, 30, 30, 3)
y_train.shape (31367,)
y_valid.shape (7842,)


In [22]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

(31367, 43)
(7842, 43)


In [23]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),

    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),

    keras.layers.Dense(43, activation='softmax')
])

In [24]:
lr = 0.001
epochs = 30

opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [25]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.5509 - loss: 1.8238 - val_accuracy: 0.9727 - val_loss: 0.0924
Epoch 2/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.9484 - loss: 0.1676 - val_accuracy: 0.9941 - val_loss: 0.0205
Epoch 3/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 41s 29ms/step - accuracy: 0.9698 - loss: 0.1047 - val_accuracy: 0.9929 - val_loss: 0.0245
Epoch 4/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - accuracy: 0.9720 - loss: 0.0917 - val_accuracy: 0.9897 - val_loss: 0.0336
Epoch 5/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.9759 - loss: 0.0792 - val_accuracy: 0.9869 - val_loss: 0.0472
Epoch 6/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.9815 - loss: 0.0595 - val_accuracy: 0.9688 - val_loss: 0.0917
Epoch 7/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.9850 - loss: 0.0491 - val_accuracy: 0.9930 - val_loss: 0.0205
Epoch 8/30
981/981 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.9829 - loss: 0.0564 - 

In [26]:
test = pd.read_csv(data_dir + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
X_test = X_test/255

pred_probabilities = model.predict(X_test)
pred = np.argmax(pred_probabilities, axis=1)


print('Test Data accuracy: ',accuracy_score(labels, pred)*100)

395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Test Data accuracy:  98.1710213776722


In [27]:
model.save('/content/model5.keras')

In [28]:
model.save('/content/model5.h5')
